## Convolution matrix, padding, and stride

In [1]:
import numpy as np

### Les fonctions

In [2]:
# padding(mat, pad)
def padding_1(mat, pad):
    if len(mat.shape)== 4:
        n_R, n_C= mat.shape[1:3]
    else:
        n_R, n_C= mat.shape

    padded_mat = np.zeros((n_R+ 2*pad, n_C+ 2*pad ))


    for i in range(n_R):
        for j in range(n_C):
            padded_mat[i+pad][j+pad] = mat[i][j]

    return padded_mat

def padding_2(mat, pad):
    if len(mat.shape) == 4:
        m, n_H, n_W, n_C = mat.shape
    else:
        n_H, n_W = mat.shape
        n_C = 1
        m=1

    padded_mat = np.zeros((m, n_H+2*pad, n_W+2*pad, n_C))

    for i in range(m):
        for j in range(n_H):
            for k in range(n_W):
                padded_mat[i, j+pad, k+pad, :] = mat[i, j, k, :]

    return padded_mat

def une_conv(mat_partie, W):
    return np.sum(mat_partie*W)

In [3]:
def conv_f(mat, W, hparameters):
    
    # Extraire les hyper-paramètres
    stride = hparameters['stride']
    pad = hparameters['pad']

    (m, n_H_p, n_W_p, n_C_p) = mat.shape
    (f, f, n_C_p, n_C) = W.shape
    
    n_H = int((n_H_p - f + 2*pad)/stride) + 1
    n_W = int((n_W_p - f + 2*pad)/stride) + 1
    
    # rembourrage
    mat = padding_2(mat, pad)
    
    # Initialisation de la matrice de sortie Z 
    Z = np.zeros((m, n_H, n_W, n_C))
    
    # Boucler sur les exemples d'entrée
    for i in range(m):
        # Extraire la matrice d'entrée i
        mat_i = mat[i,:,:,:]
        
        # Boucler sur les canaux de sortie
        for c in range(n_C):
            # Boucler sur les lignes de la matrice de sortie
            for h in range(n_H):
                # les indices de début et de fin de la ligne de la matrice d'entrée
                h_start = h*stride
                h_end = h*stride + f
                
                # Boucler sur les colonnes de la matrice de sortie
                for w in range(n_W):
                    # les indices de début et de fin de la colonne de la matrice d'entrée
                    w_start = w*stride
                    w_end = w*stride + f
                    
                    # Extraire la partie de la matrice d'entrée qui correspond au filtre actuel
                    mat_slice = mat_i[h_start:h_end, w_start:w_end, :]
                    
                    # Calcul de matrice de convolution en utilisant une_conv
                    Z[i, h, w, c] = une_conv(mat_slice, W[:, :, :, c])   
                  
    # Enregistrement des valeurs de (mat, W, hparameters)
    cache = (mat, W, hparameters)
    
    return Z, cache

### Les tests des fonctions

In [5]:
# Test de une_conv()
partie_mat = np.random.randn(3, 3, 3)
W = np.random.randn(3, 3, 3)
res = une_conv(partie_mat, W)
res

-3.799488805441893

In [6]:
# Test conv_f function
mat = np.random.randn(1, 28, 28, 3)
W = np.random.randn(3, 3, 3, 8)
hparameters = {"pad": 2, "stride": 1}
Z, cache = conv_f(mat, W, hparameters)
Z

array([[[[-0.94244584, -0.12382569, -0.50616107, ..., -1.48207773,
          -0.26765384,  0.93024992],
         [ 0.65726071,  1.17089493,  0.97367992, ..., -0.29961154,
          -1.1304008 , -0.99312619],
         [-2.12837228, -1.90357102,  0.52097094, ...,  1.07490756,
          -0.73398053, -1.20097229],
         ...,
         [-2.07449294, -1.24351805, -2.12942815, ...,  4.32082485,
           0.98285707, -2.4967404 ],
         [-0.45409174, -0.3221734 , -2.34066328, ...,  1.19358654,
           3.07780016,  1.70314411],
         [ 0.59625027,  0.04074469, -1.10747524, ..., -0.05327983,
           0.41492322,  1.44024114]],

        [[-1.65076638, -0.93948257,  1.09997564, ...,  2.55699455,
           1.04235033, -1.56033608],
         [-2.50716012, -1.51663998,  1.25095618, ..., -2.32623583,
          -2.68891568,  1.57936149],
         [ 4.32324811,  3.54243221,  5.04809454, ..., -3.79016646,
          -0.99983385, -0.03177709],
         ...,
         [ 1.55816442, -3.04514247